# Create `Dataset` for defect calculations

In this example we create inputs for vacancies, substitution and interstitials calculations in Silicon.

In [ ]:
from pynter.vasp.schemes import AdvancedSchemes
from pynter.tools.materials_project import MPDatabase

In [2]:
# we get the structure from the materials project database
structure = MPDatabase('mp-149').get_structure()
structure

Structure Summary
Lattice
    abc : 3.839943374653261 3.839943378813096 3.83994338
 angles : 59.99999998977525 59.99999995393976 60.00000000512866
 volume : 40.036809671145996
      A : 3.32548851 0.0 1.91997169
      B : 1.10849617 3.13530064 1.91997169
      C : 0.0 0.0 3.83994338
    pbc : True True True
PeriodicSite: Si (3.8797, 2.7434, 6.7199) [0.8750, 0.8750, 0.8750]
PeriodicSite: Si (0.5542, 0.3919, 0.9600) [0.1250, 0.1250, 0.1250]

### Create `AdvanceSchemes` object

The `AdvancedSchemes` class contains methods to automatically create `VaspJob` objects relative to fixed defect calculations schemes. In this case the schemes involves a two-step relaxation, first with electronic minimazation (1-PBE-SCF) and second with atomic relaxation (2-PBE-OPT). 

In [3]:
from pynter.__init__ import load_config
import os.path as op

localdir = load_config()['HPC']['localdir']
path = op.join(localdir,'tutorials/Si-defects')
# set custom job settings
job_settings = {'nodes':1,'timelimit':'02:00:00'}
# init AdvancedSchemes object
schemes = AdvancedSchemes(path=path,structure=structure,job_settings=job_settings,name='Si-test')

### Vacancies inputs

First we create vacancies jobs. We provide a dictionary with the elements of the vacancies we want to generate as keys, and the relative charge states list as values.

In [4]:
elements_with_charges = {'Si':[-1,0,1]}
jobs_vacancies = schemes.vacancies_pbe_relaxation(elements_with_charges,supercell_size=3,automation=True)

### Substitutions inputs

For substitution jobs is very similar. We provide a dictionary with `"{new_element}-on-{old_element}"` as keys, and the relative charge states list as values.

In [5]:
elements_to_replace_with_charges = {'P-on-Si':[0,1,2],'B-on-Si':[-2,-1,0]}
jobs_substitutions = schemes.substitutions_pbe_relaxation(elements_to_replace_with_charges,supercell_size=3,automation=True)

### Interstitials inputs

Since interstitials are not as straightforward as vacancies and substitutions, the `interstitials_pbe_relaxation` method has NOT been implemented, mostly to be forced to be more careful with what interstitial structures are used in the calculations. In this example, we can create `Interstitial` objects with the `create_interstitials` method, which uses `pymatgen` to find interstitial sites (using Voronoi tesselation). Since usually more than one site is found, we'll have more than one option for each interstitial, hence every case needs to be examined manually. Be aware that this could also be the case for vacancies and substitutions in certain structures with lower symmetry.

In [6]:
from pynter.defects.defects import create_interstitials
from pynter.tools.structure import view_structure_with_ase

# create Interstitial objects
interstitials = create_interstitials(structure,['Si'],supercell_size=3)
#view interstitial structures
view_structure_with_ase([i.defect_structure for i in interstitials])

The more generic `defects_pbe_relaxation` method can be used to create interstitial jobs. The input is a list of tuples, in the format `[(Defect,[q1,q2,q3,...])]`, where `q` are the desired charge states.

In [7]:
defects_with_charges = []
for inter in interstitials:
    defects_with_charges.append((inter,[-1,0,1]))
jobs_interstitials = schemes.defects_pbe_relaxation(defects_with_charges,automation=True)

## Initialize Dataset

In [8]:
from pynter.data.datasets import Dataset

# combine all jobs into one Dataset
jobs = jobs_vacancies + jobs_substitutions + jobs_interstitials
ds = Dataset(jobs)
ds.jobs_table(display=['charge'])

,formula,group,nodes,is_converged,charge
job_name,,,,,
Si-test_Int_Si(mult108)_q-1_PBE-rel_1,Si55,Int_Si(mult108),/q-1/1-PBE-SCF,None,-1.0
Si-test_Int_Si(mult108)_q-1_PBE-rel_2,Si55,Int_Si(mult108),/q-1/2-PBE-OPT,None,-1.0
Si-test_Int_Si(mult108)_q0_PBE-rel_1,Si55,Int_Si(mult108),/q0/1-PBE-SCF,None,0.0
Si-test_Int_Si(mult108)_q0_PBE-rel_2,Si55,Int_Si(mult108),/q0/2-PBE-OPT,None,0.0
Si-test_Int_Si(mult108)_q1_PBE-rel_1,Si55,Int_Si(mult108),/q1/1-PBE-SCF,None,1.0
Si-test_Int_Si(mult108)_q1_PBE-rel_2,Si55,Int_Si(mult108),/q1/2-PBE-OPT,None,1.0
Si-test_Int_Si(mult54)_q-1_PBE-rel_1,Si55,Int_Si(mult54),/q-1/1-PBE-SCF,None,-1.0
Si-test_Int_Si(mult54)_q-1_PBE-rel_2,Si55,Int_Si(mult54),/q-1/2-PBE-OPT,None,-1.0
Si-test_Int_Si(mult54)_q0_PBE-rel_1,Si55,Int_Si(mult54),/q0/1-PBE-SCF,None,0.0


In [9]:
#write input files
ds.write_jobs_input()